In [ ]:
import numpy as np

from pathlib import Path

from scipy.io import loadmat

import matplotlib.pyplot as plt

import os

scale = 2
plt.rcParams.update({
    'font.size': scale * 8,  # base font size
    'axes.labelsize': scale * 8,  # x/y label
    'xtick.labelsize': scale * 7,
    'ytick.labelsize': scale * 7,
    'legend.fontsize': scale * 5,
    'axes.titlesize': scale * 10,  # usually unused in journal figures
})
os.chdir('..')  # This changes the working directory to DiffGFDN

In [ ]:
freqs = [63, 125, 250, 500, 1000, 2000, 4000, 8000]
total_receivers = 838
ratio = np.linspace(0.3, 0.9, 7)
plt.figure()
batch_size = 32
edc_error = {}
edr_error = {}

for i in range(len(ratio)):
    edc_error[ratio[i]] = np.zeros(len(freqs))
    edr_error[ratio[i]] = np.zeros(len(freqs))
    num_receivers = np.round((1-ratio[i]) * total_receivers)
    num_batches_exact = num_receivers / batch_size
    print(num_batches_exact)
    for k in range(len(freqs)):
        file_path = Path(f'output/train_split_test/{ratio[i]:.1f}/grid_rir_treble_band_centre={freqs[k]}Hz_colorless_loss_diff_delays/individual_losses_test_loss_vs_position.mat').resolve()
        mat_dict = loadmat(file_path)
        num_batches_int = len(mat_dict['edc_loss'][0])
        assert num_batches_exact <= num_batches_int
        edc_error[ratio[i]][k] = np.sum(mat_dict['edc_loss'] / 10) / num_batches_exact
        edr_error[ratio[i]][k] =  np.sum(mat_dict['edr_loss']) / num_batches_exact
    print(f'Ratio: {ratio[i]:.1f}, EDC error : {np.round(edc_error[ratio[i]], 4)}')

    plt.semilogx(freqs, edc_error[ratio[i]], '-x', label=f'Split ratio = {ratio[i]:.1f}')

plt.ylabel('EDC error (dB)')
plt.xlabel('Frequencies (Hz)')
plt.xticks(freqs, ['63' ,'125', '250', '500', '1k', '2k', '4k', '8k'])
plt.legend()